In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install ultralytics

In [5]:
import os
import glob
import pandas as pd
import re
import math
from ultralytics import YOLO
import cv2
from PIL import Image
import matplotlib.pyplot as plt

In [6]:
# Carrega o modelo YOLO
best_model = YOLO('/kaggle/input/bestmodel/best.pt')

In [ ]:
# Função para converter tile em latitude e longitude
def tile_to_latlon(zoom, x, y):
    """Converte coordenadas de tile (z, x, y) em latitude e longitude."""
    n = 2.0 ** zoom
    lon_deg = x / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * y / n)))
    lat_deg = math.degrees(lat_rad)
    return lat_deg, lon_deg

# Função para extrair z, x, y do caminho da imagem
def extract_tile_info(image_path):
    """Extrai z, x, y do caminho da imagem e do nome do arquivo."""
    # Este padrão assume que os últimos três componentes numéricos são z, x, e y (nome do arquivo sem extensão).
    pattern = r'/(\d+)/(\d+)/([^/]+)\.png$' # Mantido como no original
    match = re.search(pattern, image_path)
    if match:
        # O terceiro grupo (nome do arquivo) é y.
        z_str, x_str, y_filename_str = match.groups()
        z = int(z_str)
        x = int(x_str)
        y = int(y_filename_str) # O nome do arquivo sem .png é o Y do tile
        return z, x, y
    else:
        raise ValueError(f"Formato de caminho inválido ou não مطابق com o padrão Z/X/Y.png: {image_path}")

# Tamanho do tile em pixels (assumindo 256x256)
tile_size = 256

# --- INÍCIO DAS MODIFICAÇÕES PRINCIPAIS ---

# Define o diretório base que contém suas pastas 
base_dir_processing = '/kaggle/input/ortofotosz21/20' 

# Nome do arquivo CSV de saída e frequência de salvamento
output_csv = '/kaggle/working/zoom20.csv'
save_every_n_folders = 100  # Salvar a cada 50 pastas X processadas

# Lista para acumular dados antes de salvar no CSV
all_data_for_csv = []
processed_x_folders_count = 0 # Contador para as pastas X processadas

print(f"Procurando subdiretórios (pastas X) em: {base_dir_processing}")

# Lista todas as subpastas (pastas X) diretamente dentro de base_dir_processing
try:
    # Garante que estamos listando apenas diretórios
    x_folders = [f.path for f in os.scandir(base_dir_processing) if f.is_dir()]
except FileNotFoundError:
    print(f"Erro Crítico: Diretório base de processamento '{base_dir_processing}' não encontrado. Verifique o caminho.")
    x_folders = [] # Define como lista vazia para evitar mais erros

if not x_folders:
    print(f"Nenhuma subpasta (pasta X) encontrada em '{base_dir_processing}'. Verifique o caminho e a estrutura de pastas.")
    # Considere adicionar exit() se não houver pastas para processar
else:
    print(f"Encontradas {len(x_folders)} subpastas (pastas X) para processar.")

total_x_folders = len(x_folders)

# Itera sobre cada pasta X
for folder_index, x_folder_path in enumerate(x_folders):
    current_folder_num = folder_index + 1
    print(f"\nProcessando pasta X {current_folder_num}/{total_x_folders}: {x_folder_path}")

    # Lista todas as imagens PNG dentro da pasta X atual
    image_paths_in_x_folder = glob.glob(os.path.join(x_folder_path, '*.png'))

    if not image_paths_in_x_folder:
        print(f"  Nenhuma imagem .png encontrada em '{x_folder_path}'. Pulando para a próxima pasta X.")
        # Continua para a próxima pasta X, mas ainda conta esta pasta para o salvamento periódico
    else:
        print(f"  Encontradas {len(image_paths_in_x_folder)} imagens PNG para processar em '{x_folder_path}'.")

    # Itera sobre todas as imagens na pasta X atual
    for image_path in image_paths_in_x_folder:
        try:
            # Realiza a predição
            results = best_model.predict(image_path, conf=0.45, imgsz=256, verbose=False) # verbose=False para menos output

            # Extrai z, x, y do caminho da imagem
            # Note que 'extract_tile_info' agora é chamado aqui, dentro do try-except
            z, x, y = extract_tile_info(image_path)
            tile_lat, tile_lon = tile_to_latlon(z, x, y)

            if not results or not results[0].boxes or len(results[0].boxes) == 0:
                # print(f"    Nenhuma detecção para a imagem {image_path}.") # Descomente se precisar deste log
                continue # Pula para a próxima imagem se não houver detecções

            # Extrai as coordenadas das caixas delimitadoras
            for i, result_box in enumerate(results[0].boxes):
                box_coords = result_box.xyxy[0].cpu().numpy()
                x_min, y_min, x_max, y_max = box_coords
                class_id = int(result_box.cls.cpu().numpy().item())
                class_name = results[0].names[class_id]

                vertices = [
                    (x_min, y_min), (x_max, y_min),
                    (x_min, y_max), (x_max, y_max)
                ]
                vertex_coords = []
                for vx, vy in vertices:
                    x_offset = vx / tile_size
                    y_offset = vy / tile_size
                    x_adjusted = x + x_offset
                    y_adjusted = y + y_offset
                    v_lat, v_lon = tile_to_latlon(z, x_adjusted, y_adjusted)
                    vertex_coords.append((v_lat, v_lon))

                (top_left_lat, top_left_lon), (top_right_lat, top_right_lon), \
                (bottom_left_lat, bottom_left_lon), (bottom_right_lat, bottom_right_lon) = vertex_coords

                all_data_for_csv.append({
                    'image_path': image_path,
                    'object_id': i + 1,
                    'class': class_name,
                    'x_min': x_min, 'y_min': y_min, 'x_max': x_max, 'y_max': y_max,
                    'tile_latitude': tile_lat, 'tile_longitude': tile_lon,
                    'top_left_latitude': top_left_lat, 'top_left_longitude': top_left_lon,
                    'top_right_latitude': top_right_lat, 'top_right_longitude': top_right_lon,
                    'bottom_left_latitude': bottom_left_lat, 'bottom_left_longitude': bottom_left_lon,
                    'bottom_right_latitude': bottom_right_lat, 'bottom_right_longitude': bottom_right_lon
                })
        except ValueError as e_tile_info: # Erro do extract_tile_info
            print(f"    Erro ao extrair informações do tile de '{image_path}': {e_tile_info}. Pulando esta imagem.")
            continue
        except Exception as e_predict: # Outros erros (ex: YOLO, processamento de resultados)
            print(f"    Erro ao processar a imagem '{image_path}': {e_predict}. Pulando esta imagem.")
            continue
    # Fim do loop de imagens na pasta X atual

    processed_x_folders_count += 1

    # Salva a cada 'save_every_n_folders' OU se for a última pasta X E houver dados para salvar
    if (processed_x_folders_count % save_every_n_folders == 0) or (current_folder_num == total_x_folders):
        if all_data_for_csv: # Só salva se houver dados acumulados
            print(f"  Salvando dados de {len(all_data_for_csv)} detecções no CSV (total de pastas X processadas: {processed_x_folders_count})...")
            df_to_save = pd.DataFrame(all_data_for_csv)
            
            # Verifica se o arquivo CSV já existe para decidir sobre o cabeçalho
            file_exists = os.path.exists(output_csv)
            df_to_save.to_csv(output_csv, mode='a', header=not file_exists, index=False)
            
            print(f"  Dados apensados em: {output_csv}")
            all_data_for_csv = [] # Limpa a lista para liberar memória para o próximo lote
        elif (current_folder_num == total_x_folders):
             print(f"  Nenhum dado novo para salvar no CSV ao final do processamento das pastas X.")
        else:
            print(f"  Nenhum dado novo para salvar no CSV neste lote de {save_every_n_folders} pastas X (total processadas: {processed_x_folders_count}).")

# --- FIM DAS MODIFICAÇÕES PRINCIPAIS ---

print("\nProcessamento de todas as pastas X concluído.")
if os.path.exists(output_csv):
    print(f"O arquivo CSV final está salvo em: {output_csv}")
else:
    print("Nenhum dado foi salvo no CSV durante a execução.")

Procurando subdiretórios (pastas X) em: /kaggle/input/ortofotosz21/20
Encontradas 254 subpastas (pastas X) para processar.

Processando pasta X 1/254: /kaggle/input/ortofotosz21/20/407657
  Encontradas 367 imagens PNG para processar em '/kaggle/input/ortofotosz21/20/407657'.

Processando pasta X 2/254: /kaggle/input/ortofotosz21/20/407599
  Encontradas 367 imagens PNG para processar em '/kaggle/input/ortofotosz21/20/407599'.

Processando pasta X 3/254: /kaggle/input/ortofotosz21/20/407546
  Encontradas 367 imagens PNG para processar em '/kaggle/input/ortofotosz21/20/407546'.

Processando pasta X 4/254: /kaggle/input/ortofotosz21/20/407765
  Encontradas 367 imagens PNG para processar em '/kaggle/input/ortofotosz21/20/407765'.

Processando pasta X 5/254: /kaggle/input/ortofotosz21/20/407552
  Encontradas 367 imagens PNG para processar em '/kaggle/input/ortofotosz21/20/407552'.

Processando pasta X 6/254: /kaggle/input/ortofotosz21/20/407668
  Encontradas 367 imagens PNG para processar em

In [8]:
import os

def encontrar_pastas_vazias(diretorio_pai):
    """
    Analisa um diretório pai e retorna uma lista de pastas vazias.

    Args:
        diretorio_pai (str): O caminho para o diretório a ser analisado.

    Returns:
        list: Uma lista contendo os nomes das pastas vazias.
              Retorna None se o diretorio_pai não existir.
    """
    if not os.path.isdir(diretorio_pai):
        print(f"Erro: O diretório '{diretorio_pai}' não foi encontrado.")
        print("Por favor, verifique o caminho no ambiente Kaggle.")
        return None

    pastas_vazias = []
    # Itera sobre os itens no diretorio_pai
    for nome_item in os.listdir(diretorio_pai):
        caminho_completo_item = os.path.join(diretorio_pai, nome_item)
        # Verifica se o item é um diretório
        if os.path.isdir(caminho_completo_item):
            # Verifica se o diretório (subpasta) está vazio
            if not os.listdir(caminho_completo_item):
                pastas_vazias.append(nome_item) # Adiciona o nome da subpasta vazia
    return pastas_vazias

# --- COMO USAR NO KAGGLE ---

# 1. DESCUBRA O CAMINHO CORRETO PARA A PASTA "21":
#    Você pode usar o painel "Data" à direita no editor de notebooks do Kaggle
#    para ver a estrutura dos seus datasets e copiar o caminho.
#    Ou, em uma célula de código, execute: !ls /kaggle/input/ NOME_DO_SEU_DATASET/
#    para listar o conteúdo do seu dataset e encontrar a pasta "21".

#    Exemplos de como o caminho pode se parecer:
#    Se "21" está na raiz de um dataset chamado "meu-dataset-de-imagens":
#    diretorio_principal = '/kaggle/input/meu-dataset-de-imagens/21'

#    Se "21" está dentro de outras pastas no seu dataset, por exemplo:
#    Dataset: "meu-dataset-de-imagens"
#    Estrutura: meu-dataset-de-imagens -> bestmodel -> zoom21 -> 21
#    Então o caminho seria:
#    diretorio_principal = '/kaggle/input/meu-dataset-de-imagens/bestmodel/zoom21/21'

#    Se você criou a pasta "21" em /kaggle/working/:
#    diretorio_principal = '/kaggle/working/21'
#    ou
#    diretorio_principal = '/kaggle/working/Datasets/bestmodel/zoom21/21' # Se você recriou a estrutura

# ⬇️ MODIFIQUE A LINHA ABAIXO COM O CAMINHO CORRETO NO SEU NOTEBOOK KAGGLE ⬇️
diretorio_principal = '/kaggle/input/ortofotosz21/20' # ⬅️ AJUSTE AQUI!

print(f"Analisando o diretório: {diretorio_principal}")

pastas_sem_processamento = encontrar_pastas_vazias(diretorio_principal)

if pastas_sem_processamento is not None:
    if pastas_sem_processamento:
        print("\nPastas (dentro de '21') que parecem estar vazias (sem imagens/arquivos):")
        for pasta in pastas_sem_processamento:
            print(f"- {pasta}")
    else:
        print("\nNenhuma subpasta vazia encontrada dentro de '21'. Todas parecem conter arquivos. 🎉")

Analisando o diretório: /kaggle/input/ortofotosz21/20

Nenhuma subpasta vazia encontrada dentro de '21'. Todas parecem conter arquivos. 🎉
